In [1]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
%matplotlib inline

In [2]:
train_df = pd.read_csv('./titanicTrain.csv')
test_df = pd.read_csv('./titanicQuestion.csv')

In [3]:
train_df = train_df[:1000]
print('Train:', len(train_df),
      'Test:', len(test_df))

Train: 1000 Test: 309


## preprocessing function

In [131]:
from sklearn import preprocessing
# Titanic data preprocessing function
def preprocessTitanicData(raw_df):

    df = raw_df[['survived', 'pclass', 'sibsp', 'parch', 'age', 'sex', 'fare', 'embarked']]
    
    df['age'] = df['age'].fillna(df['age'].mean())
    df['fare'] = df['fare'].fillna(df['fare'].mean())
    df['sex'] = df['sex'].map({'female':0, 'male':1}).astype(int)
    df['family_size'] = df['sibsp'] + df['parch']

    x_onehot_df = pd.get_dummies(data=df, columns= ['embarked'])
    ndarray = x_onehot_df.values
    feature = ndarray[:,1:]
    label = ndarray[:, 0]
    
    minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))
    scaledFeatures = minmax_scale.fit_transform(feature)
    
    return scaledFeatures, label

In [132]:
train_df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,prediction
0,1.0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO",0
1,1.0,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON",0
2,1.0,0.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",0
3,1.0,0.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON",0
4,1.0,0.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",0


In [133]:
x_train, y_train = preprocessTitanicData(train_df)
x_test, y_test = preprocessTitanicData(test_df)

D:\andyl\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
D:\andyl\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\andyl\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vi

In [134]:
print(x_train.shape, y_train.shape) #np.array np.array
print(x_test.shape, y_test.shape) # np.array np.array

# reshape y_train and y_test
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

print(x_train.shape, y_train.shape) #np.array np.array
print(x_test.shape, y_test.shape) # np.array np.array

(1000, 10) (1000,)
(309, 10) (309,)
(1000, 10) (1000, 1)
(309, 10) (309, 1)


## construct model

In [135]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision.transforms as transforms

In [136]:
DROPOUT_RATIO = 0.5

In [154]:
class LR(nn.Module): # 繼承torch.nn.module特性
    
    # 架構
    def __init__(self):
        super(LR, self).__init__() # super(LR, self) 繼承自己的__init__()
        
        
        self.dropout = nn.Dropout(DROPOUT_RATIO)
        self.hidden1 = nn.Linear(10, 200) # input 10, hidden 200 Linear全連接
        self.hidden2 = nn.Linear(200, 150) # input 200, hidden 150  Linear全連接
       # self.hidden22 = nn.Linear(100, 40)
        self.hidden3 = nn.Linear(150, 75) # input 150, hidden 75  Linear全連接
       # self.hidden22 = nn.Linear(100, 40)
        self.hidden4 = nn.Linear(75, 50) # input 75, hidden 50  Linear全連接
       # self.hidden3 = nn.Bilinear(40, 40, 30)
        self.hidden5 = nn.Linear(50, 25) # input 50, hidden 25  Linear全連接
        self.hidden6 = nn.Linear(25, 1) # hidden 50, output 1

    # 運作流程（資料流）
    def forward(self, x):  
        x = F.sigmoid(self.hidden1(x)) # 讓x由hidden1後，進入activation function
        x = F.dropout(self.dropout(x))
        
        #y = F.relu(self.hidden22(x))
        #y = F.dropout(self.dropout(y))
        
        x = F.relu(self.hidden2(x))
        x = F.dropout(self.dropout(x))
        
        x = F.relu(self.hidden3(x))
       # x = F.relu(self.hidden3(x, y))
        x = F.dropout(self.dropout(x))
        x = F.sigmoid(self.hidden4(x))
        x = F.dropout(self.dropout(x))
        x = F.sigmoid(self.hidden5(x))
        out = self.hidden6(x)
        return out
    

In [155]:
LEARNING_RATE = 0.001
model = LR()
loss_func = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE) #model.parameters() 所有訓練的參數都在這
# model.cuda()   

## Training

In [156]:
epochs = 3000

In [157]:
for epoch in range(epochs):
    epoch += 1

    # convert to variables
    x = Variable(torch.from_numpy(x_train).float())
    y = Variable(torch.from_numpy(y_train).float())

    # clear gradient w.r.t. parameters 
    optimizer.zero_grad()
    
    # forward to get output
    prediction = model(x)

    # calculate loss
    loss = loss_func(prediction, y)

    # backward to get gradient
    loss.backward()

    # update parameters
    optimizer.step() 

    if epoch % 10 == 0:
    # plot and show learning process
        print("epoch %d, loss %.8f" % (epoch, loss.data[0]))

D:\andyl\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch 10, loss 0.25067335
epoch 20, loss 0.25107718
epoch 30, loss 0.24896705
epoch 40, loss 0.24710101
epoch 50, loss 0.24620938
epoch 60, loss 0.24547240
epoch 70, loss 0.24835649
epoch 80, loss 0.24585992
epoch 90, loss 0.24436615
epoch 100, loss 0.24589597
epoch 110, loss 0.24770850
epoch 120, loss 0.24312043
epoch 130, loss 0.24600756
epoch 140, loss 0.24639918
epoch 150, loss 0.24481127
epoch 160, loss 0.24652623
epoch 170, loss 0.24543175
epoch 180, loss 0.24341340
epoch 190, loss 0.24558924
epoch 200, loss 0.24440400
epoch 210, loss 0.24414571
epoch 220, loss 0.24408019
epoch 230, loss 0.24645658
epoch 240, loss 0.24446276
epoch 250, loss 0.24549700
epoch 260, loss 0.24395025
epoch 270, loss 0.24439621
epoch 280, loss 0.24530423
epoch 290, loss 0.24457288
epoch 300, loss 0.24410102
epoch 310, loss 0.24593040
epoch 320, loss 0.24669911
epoch 330, loss 0.24546573
epoch 340, loss 0.24668485
epoch 350, loss 0.24391961
epoch 360, loss 0.24548465
epoch 370, loss 0.24576490
epoch 380,

epoch 2980, loss 0.24337885
epoch 2990, loss 0.24474610
epoch 3000, loss 0.24616246


In [158]:
# get filters from parameters:
print(list(model.parameters())[0].size())  

torch.Size([200, 10])


In [159]:
train_prediction = prediction.data.numpy()
train_prediction = np.where(train_prediction>=0.7,1,0)
train_df['prediction'] = train_prediction

## Prediction

In [160]:
x_test_v = Variable(torch.from_numpy(x_test).float())
test_prediction = model(x_test_v).data.numpy()
# test_prediction = np.where(test_prediction>=0.6,1,0)

In [161]:
test_df['prediction'] = test_prediction

In [162]:
def apply_sub(row, label):
    if row[label] >= 0.5:
        return 1
    else:
        return 0

test_df['prediction'] = test_df.apply(lambda x: apply_sub(x, 'prediction'), axis = 1)

## Verify model

In [163]:
answer = pd.read_csv('titanicAns.csv')
accuracy = 1.0 - sum(abs(answer['survived'] - test_df['prediction'])) / test_df.shape[0]
print('Accuracy: %f %%'%(accuracy*100))

Accuracy: 75.404531 %


### dump accuracy

In [164]:
with open('log.txt', 'a') as flog:
    flog.write('RESULT:\n\tDropout:\t%.3f\n\tepochs:\n\t%ilearning_rate:\t%f\n\tAccuracy:\t%f %%\n'%(DROPOUT_RATIO, epochs, learning_rate, accuracy*100))

In [165]:
with open('log.txt', 'r') as flog:
    print(flog.read())

RESULT:
	Dropout:	0.100
	epochs:		5000
	Accuracy:	70.550162 %
RESULT:
	Dropout:	0.300
	epochs:		5000
	Accuracy:	73.462783 %
RESULT:
	Dropout:	0.500
	epochs:		5000
	Accuracy:	72.815534 %
RESULT:
	Dropout:	0.350
	epochs:		5000
	learning_rate:	0.03
	Accuracy:	73.462783 %
RESULT:
	Dropout:	0.350
	epochs:		5000
	learning_rate:	0.030000
	Accuracy:	67.961165 %
RESULT:
	Dropout:	0.350
	epochs:		3000
	learning_rate:	0.003000
	Accuracy:	75.080906 %
RESULT:
	Dropout:	0.500
	epochs:		3000
	learning_rate:	0.003000
	Accuracy:	74.757282 %
RESULT:
	Dropout:	0.500
	epochs:		3000
	learning_rate:	0.003000
	Accuracy:	75.080906 %
RESULT:
	Dropout:	0.500
	epochs:
	3000learning_rate:	0.003000
	Accuracy:	75.080906 %
RESULT:
	Dropout:	0.500
	epochs:
	3000learning_rate:	0.003000
	Accuracy:	74.110032 %
RESULT:
	Dropout:	0.500
	epochs:
	3000learning_rate:	0.003000
	Accuracy:	75.404531 %

